In [73]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [74]:
df = pd.read_csv('train.csv')
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [75]:
df.isnull().mean() *100

Survived     0.000000
Pclass       0.000000
Sex          0.000000
Age         19.865320
SibSp        0.000000
Parch        0.000000
Fare         0.000000
Embarked     0.224467
dtype: float64

In [76]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [77]:
X_train, X_test , y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [78]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
30,1,male,40.0,0,0,27.7208,C
10,3,female,4.0,1,1,16.7000,S
873,3,male,47.0,0,0,9.0000,S
182,3,male,9.0,4,2,31.3875,S
876,3,male,20.0,0,0,9.8458,S
...,...,...,...,...,...,...,...
534,3,female,30.0,0,0,8.6625,S
584,3,male,NaN,0,0,8.7125,C
493,1,male,71.0,0,0,49.5042,C
527,1,male,NaN,0,0,221.7792,S


In [79]:
X_train.isnull().mean()

Pclass      0.000000
Sex         0.000000
Age         0.207865
SibSp       0.000000
Parch       0.000000
Fare        0.000000
Embarked    0.002809
dtype: float64

# Imputer + Scaling

In [80]:
numerical_features = ['Age','Fare']
numerical_transformer = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='mean')),
        ('scaler',StandardScaler())
    ]
)


categorical_features = ['Sex','Embarked']
categorical_transformer = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ohe',OneHotEncoder(handle_unknown='ignore'))
    ]
)

# Using Transformer

In [81]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num',numerical_transformer,numerical_features),
        ('cat',categorical_transformer,categorical_features)
    ],remainder='passthrough'
)

# Logistic regression

In [82]:
clf = Pipeline(
    steps=[
        ('preprocessor',preprocessor),
        ('classifier',LogisticRegression())
    ]
)

# Diagram Draw

In [83]:
from sklearn import set_config

set_config(display='diagram')
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked'])])),
                ('classifier', LogisticRegression())])

In [84]:
from sklearn.preprocessing import MinMaxScaler
param_grid = {
    'preprocessor__num__imputer__strategy' : ['mean','meadian'],
    'preprocessor__num__scaler' : [StandardScaler(),MinMaxScaler()],
    'preprocessor__cat__imputer__strategy' : ['most_frequent','constant'],
    'classifier__C': [0.1, 1.0, 10, 100]
}

grid_search = GridSearchCV(clf,param_grid,cv=10)

In [85]:
grid_search.fit(X_train, y_train)

print(f"Best params: {grid_search.best_params_}")

Best params: {'classifier__C': 0.1, 'preprocessor__cat__imputer__strategy': 'most_frequent', 'preprocessor__num__imputer__strategy': 'mean', 'preprocessor__num__scaler': StandardScaler()}


C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
160 fits failed out of a total of 320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
160 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 359, in _fit
    

In [86]:
print(f"Internal CV score {grid_search.best_score_}")

Internal CV score 0.8117175273865416


In [87]:
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results.sort_values("mean_test_score",ascending=False)
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,param_preprocessor__num__scaler,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009013,6.445429e-04,0.003423,0.000501,0.1,most_frequent,mean,StandardScaler(),"{'classifier__C': 0.1, 'preprocessor__cat__imp...",0.791667,...,0.845070,0.788732,0.816901,0.816901,0.774648,0.774648,0.816901,0.811718,0.033025,1
4,0.007802,4.000664e-04,0.002701,0.000459,0.1,constant,mean,StandardScaler(),"{'classifier__C': 0.1, 'preprocessor__cat__imp...",0.791667,...,0.830986,0.788732,0.816901,0.816901,0.774648,0.774648,0.816901,0.810309,0.031852,2
8,0.008409,6.699075e-04,0.002703,0.000460,1.0,most_frequent,mean,StandardScaler(),"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.791667,...,0.816901,0.774648,0.816901,0.788732,0.774648,0.774648,0.830986,0.806084,0.033459,3
12,0.009034,5.875694e-04,0.002801,0.000400,1.0,constant,mean,StandardScaler(),"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.791667,...,0.816901,0.774648,0.816901,0.788732,0.774648,0.774648,0.830986,0.806084,0.033459,3
17,0.009702,6.403517e-04,0.002801,0.000400,10,most_frequent,mean,MinMaxScaler(),"{'classifier__C': 10, 'preprocessor__cat__impu...",0.791667,...,0.816901,0.774648,0.830986,0.788732,0.774648,0.760563,0.830986,0.806084,0.035752,5
21,0.011703,9.001970e-04,0.002901,0.000539,10,constant,mean,MinMaxScaler(),"{'classifier__C': 10, 'preprocessor__cat__impu...",0.777778,...,0.816901,0.774648,0.830986,0.788732,0.774648,0.760563,0.830986,0.804695,0.036546,6
16,0.008502,5.001784e-04,0.002601,0.000490,10,most_frequent,mean,StandardScaler(),"{'classifier__C': 10, 'preprocessor__cat__impu...",0.791667,...,0.802817,0.774648,0.830986,0.788732,0.774648,0.760563,0.830986,0.804675,0.035575,7
9,0.008807,4.030255e-04,0.002702,0.000459,1.0,most_frequent,mean,MinMaxScaler(),"{'classifier__C': 1.0, 'preprocessor__cat__imp...",0.805556,...,0.816901,0.788732,0.802817,0.802817,0.760563,0.774648,0.802817,0.804656,0.032183,8
20,0.009908,5.301616e-04,0.002901,0.000539,10,constant,mean,StandardScaler(),"{'classifier__C': 10, 'preprocessor__cat__impu...",0.777778,...,0.802817,0.774648,0.830986,0.788732,0.774648,0.760563,0.830986,0.803286,0.036319,9
24,0.009701,4.580485e-04,0.003201,0.000400,100,most_frequent,mean,StandardScaler(),"{'classifier__C': 100, 'preprocessor__cat__imp...",0.777778,...,0.802817,0.774648,0.830986,0.788732,0.774648,0.760563,0.830986,0.803286,0.036319,9


In [89]:
cv_results[['param_classifier__C','param_preprocessor__cat__imputer__strategy','param_preprocessor__num__imputer__strategy','param_preprocessor__num__scaler','mean_test_score']]

,param_classifier__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,param_preprocessor__num__scaler,mean_test_score
0,0.1,most_frequent,mean,StandardScaler(),0.811718
4,0.1,constant,mean,StandardScaler(),0.810309
8,1.0,most_frequent,mean,StandardScaler(),0.806084
12,1.0,constant,mean,StandardScaler(),0.806084
17,10,most_frequent,mean,MinMaxScaler(),0.806084
21,10,constant,mean,MinMaxScaler(),0.804695
16,10,most_frequent,mean,StandardScaler(),0.804675
9,1.0,most_frequent,mean,MinMaxScaler(),0.804656
20,10,constant,mean,StandardScaler(),0.803286
24,100,most_frequent,mean,StandardScaler(),0.803286
